# Spec-Kit Voxel Pipeline (Colab)

This notebook sets up the environment and runs the voxelization pipeline using the code from the GitHub repository `yamatotakeru616/spec-kit`.

In [ ]:
# 1. Mount Google Drive
# This allows access to your files in Google Drive.
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# 2. Clone Repository & Install Dependencies
# Clones the latest code from GitHub and installs necessary Python packages.

import os
import shutil

# FORCE FRESH CLONE: Delete existing folder to avoid errors
if os.path.exists('/content/spec-kit'):
    shutil.rmtree('/content/spec-kit')

!git clone https://github.com/yamatotakeru616/spec-kit.git /content/spec-kit
%cd /content/spec-kit
!pip install -r requirements.txt

# Verify installation
import numba
from numba import cuda
print(f"Numba version: {numba.__version__}")
print(f"CUDA available: {cuda.is_available()}")
if not cuda.is_available():
    print("WARNING: CUDA is not available. Please change the Runtime type to GPU (Runtime > Change runtime type > T4 GPU) for faster processing.")

In [ ]:
# 3. Setup Paths & Workspace
import sys
import os

# Add src to path so we can import modules
sys.path.append('/content/spec-kit')
sys.path.append('/content/spec-kit/src')

from src import loader, utils, voxel_cuda, classify, io_helpers

# Define Workspace directories in Google Drive
WORKDIR = "/content/drive/MyDrive/voxel_engine"
INPUT_DIR = os.path.join(WORKDIR, "input")
OUTPUT_DIR = os.path.join(WORKDIR, "output")

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Workspace setup at: {WORKDIR}")
print(f"Input folder: {INPUT_DIR}")
print(f"Output folder: {OUTPUT_DIR}")
print("\nFiles in Input folder:")
!ls "$INPUT_DIR"

In [ ]:
# 4. Run Voxelization Pipeline

# --- CONFIGURATION ---
TARGET_FILENAME = "example.laz" # <--- CHANGE THIS to your uploaded file name
VOXEL_PITCH = 0.5               # Voxel size in meters
VOXEL_THRESH = 1                # Minimum points per voxel
# ---------------------

target_path = os.path.join(INPUT_DIR, TARGET_FILENAME)

if os.path.exists(target_path):
    print(f"Processing: {target_path} ...")
    
    # 1. Load Point Cloud
    try:
        points, colors = loader.load_pointcloud(target_path)
        print(f"Loaded {points.shape[0]} points.")
    except Exception as e:
        print(f"Error loading file: {e}")
        points = None

    if points is not None:
        # 2. Voxelize
        if voxel_cuda.has_cuda:
            print("Running CUDA Voxelization...")
            voxels, counts, origin, pitch_out = voxel_cuda.voxelize_cuda(points, pitch=VOXEL_PITCH, thresh=VOXEL_THRESH)
        else:
            print("Running CPU Voxelization (might be slow)...")
            voxels, counts, origin, pitch_out = voxel_cuda.voxelize_cpu(points, pitch=VOXEL_PITCH, thresh=VOXEL_THRESH)
            
        print(f"Voxelization complete. Grid shape: {voxels.shape}")
        
        # 3. Save Output (Example: NPY)
        # You can extend this to use io_helpers for other formats
        output_filename = f"{os.path.splitext(TARGET_FILENAME)[0]}_voxels.npy"
        output_path = os.path.join(OUTPUT_DIR, output_filename)
        utils.save_npy(output_path, voxels)
        print(f"Saved voxel data to: {output_path}")
        
else:
    print(f"[!] File not found: {TARGET_FILENAME}")
    print(f"Please upload your point cloud file to: {INPUT_DIR}")